In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

In [2]:
pf = pd.read_csv('data/FEdata.csv')

In [3]:
pf = pf.drop(['Over18','StandardHours'],axis=1)

In [4]:
pf.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,37,0,2,1,1,4,1,77,1,1,...,3,3,1,7,2,4,7,5,0,7
1,54,0,1,1,1,4,1,1245,4,0,...,3,1,1,33,2,1,5,4,1,4
2,34,1,1,1,7,3,1,147,1,1,...,4,4,0,9,3,3,9,7,0,6
3,39,0,2,1,1,1,1,1026,4,0,...,3,3,1,21,3,3,21,6,11,8
4,28,1,1,1,1,3,3,1111,1,1,...,3,1,2,1,2,3,1,0,0,0


In [5]:
pf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 29 columns):
Age                         1100 non-null int64
Attrition                   1100 non-null int64
BusinessTravel              1100 non-null int64
Department                  1100 non-null int64
DistanceFromHome            1100 non-null int64
Education                   1100 non-null int64
EducationField              1100 non-null int64
EmployeeNumber              1100 non-null int64
EnvironmentSatisfaction     1100 non-null int64
Gender                      1100 non-null int64
JobInvolvement              1100 non-null int64
JobLevel                    1100 non-null int64
JobRole                     1100 non-null int64
JobSatisfaction             1100 non-null int64
MaritalStatus               1100 non-null int64
MonthlyIncome               1100 non-null int64
NumCompaniesWorked          1100 non-null int64
OverTime                    1100 non-null int64
PercentSalaryHike           1

In [6]:
y_train = torch.from_numpy(np.array(pf['Attrition']))

In [7]:
y_train.type()

'torch.LongTensor'

In [8]:
y_train.size()

torch.Size([1100])

In [9]:
y_train = y_train.reshape(1100,1)

In [10]:
y_train = y_train.float()

In [11]:
pf = pf.drop(['Attrition'],axis=1)

In [12]:
pfnp = np.array(pf)

In [13]:
pfnp.dtype

dtype('int64')

In [14]:
pfnp = pfnp/pfnp.max(axis=0)
pfnp

array([[0.61666667, 1.        , 0.5       , ..., 0.27777778, 0.        ,
        0.41176471],
       [0.9       , 0.5       , 0.5       , ..., 0.22222222, 0.06666667,
        0.23529412],
       [0.56666667, 0.5       , 0.5       , ..., 0.38888889, 0.        ,
        0.35294118],
       ...,
       [0.61666667, 1.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.36666667, 1.        , 0.5       , ..., 0.        , 0.        ,
        0.        ],
       [0.43333333, 0.5       , 0.5       , ..., 0.11111111, 0.06666667,
        0.11764706]])

In [15]:
x_train = torch.from_numpy(pfnp)

In [16]:
x_train.size()

torch.Size([1100, 28])

In [17]:
x_train = x_train.float()
x_train.type()

'torch.FloatTensor'

In [18]:
x_train

tensor([[0.6167, 1.0000, 0.5000,  ..., 0.2778, 0.0000, 0.4118],
        [0.9000, 0.5000, 0.5000,  ..., 0.2222, 0.0667, 0.2353],
        [0.5667, 0.5000, 0.5000,  ..., 0.3889, 0.0000, 0.3529],
        ...,
        [0.6167, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3667, 1.0000, 0.5000,  ..., 0.0000, 0.0000, 0.0000],
        [0.4333, 0.5000, 0.5000,  ..., 0.1111, 0.0667, 0.1176]])

In [19]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(28,128)
        self.l2 = nn.Linear(128,32)
        self.l3 = nn.Linear(32,1)
        
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = torch.sigmoid(self.l3(x))
        return x
    
net = Net()

print(net)

Net(
  (l1): Linear(in_features=28, out_features=128, bias=True)
  (l2): Linear(in_features=128, out_features=32, bias=True)
  (l3): Linear(in_features=32, out_features=1, bias=True)
)


In [21]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
loss_func = torch.nn.MSELoss() 

for i in range(1000):
    out = net(x_train)
    loss = loss_func(out,y_train)
    optimizer.zero_grad()   # 清空上一步的残余更新参数值
    loss.backward()         # 误差反向传播, 计算参数更新值
    optimizer.step()        # 将参数更新值施加到 net 的 parameters 上

In [23]:
pf_pre = pd.read_csv('data/FEtest.csv')
pf_pre = pf_pre.drop(['Over18','StandardHours'],axis=1)
pfnp_pre = np.array(pf_pre)
pfnp_pre = pfnp_pre/pfnp_pre.max(axis=0)
x_pre = torch.from_numpy(pfnp_pre)
x_pre.size()

torch.Size([350, 28])

In [27]:
x_pre = x_pre.float()
pre = net(x_pre)

In [ ]:
pre

In [28]:
pre = pre.detach().numpy()

In [29]:
len(pre)

350

In [30]:
for x in range(len(pre)):
    if pre[x]>0.5:
        pre[x]=1
    else:
        pre[x]=0
pre

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],

In [31]:
out = pd.DataFrame(pre,dtype='int64')

In [32]:
out.head(1100)

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [33]:
out.columns=['result']

In [34]:
out.head()

,result
0,0
1,0
2,0
3,0
4,0


In [36]:
out.to_csv('result2.csv',index=False)

In [3]:
corrDf = pf.corr()
corrDf['Attrition'].sort_values(ascending =False)

Attrition                   1.000000
OverTime                    0.267080
DistanceFromHome            0.088563
JobRole                     0.086899
EducationField              0.055239
Department                  0.053364
PerformanceRating           0.046762
PercentSalaryHike           0.026604
NumCompaniesWorked          0.025889
Gender                      0.016750
BusinessTravel              0.015483
TrainingTimesLastYear      -0.043395
EmployeeNumber             -0.045168
Education                  -0.046494
WorkLifeBalance            -0.048794
RelationshipSatisfaction   -0.051749
YearsSinceLastPromotion    -0.071760
EnvironmentSatisfaction    -0.097003
JobInvolvement             -0.122722
JobSatisfaction            -0.125568
StockOptionLevel           -0.138498
YearsAtCompany             -0.143697
MaritalStatus              -0.153045
MonthlyIncome              -0.155521
YearsWithCurrManager       -0.158558
YearsInCurrentRole         -0.163059
JobLevel                   -0.168775
A